This is a script for calculating playoff probabilities in a fantasy sports league that is (1) points-based and (2) head-to-head. In this latest update, it no longer requires an outside excel document with information (although to save time, you may want to save the schedule dataframe externally as opposed to scraping that every time you run the script).

You'll need to replace a few references to URLs with your own league + install selenium to use this. That may be straightforward (just install the requisite packages and let the machine do the rest) but this isn't my first project using this package.

If your points system has a high probability for ties or a complex tiebreaker system you may want to augment this code to include that possibility. I treat ties as 0.5 wins.

I'm leaving in the URLs and team names, there's no PII, I'm not telling you which team is mine, and none of the names are even offensive, so I'm not super worried about privacy in this context.

**5.0 Update Notes (12.25.22)**

1. Massively overhauled structure of "does the deal" loops so that we're not appending to pandas until the end
2. Web scraping integration of schedule-reading and score-scraping, no separate manual file required now (also now resilient to team name changes)

**5.1 Update Notes (8.25.23)**

1. Looks like selenium went and cleaned itself up, code reflects that
2. FINALLY handling logins in the selenium code instead of handling manually in browser
3. The latest league I'm doing this for is 14 teams so misc changes to handle that

In [67]:
#ONLY MANUAL UPDATE REQUIRED

wk_count = 1
league_url = 'https://football.fantasysports.yahoo.com/f1/664138'

## Package Imports & Selenium Init

In [66]:
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
import collections
from functools import reduce

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import time

pd.set_option('display.max_columns', 30)

In [3]:
service = Service('chromedriver')
driver = webdriver.Chrome()
driver.get(league_url)

In [7]:
#you would need to input your own username or login in the send_keys functions (or, preferably, import them from elsewhere)

driver.get(league_url)

login = driver.find_element(By.XPATH, '//*[@id="login-username"]')
login.send_keys(" ")

clickable = driver.find_element(By.ID, 'login-signin')
clickable.click()

time.sleep(3)

pw = driver.find_element(By.XPATH, '//*[@id="login-passwd"]')
pw.send_keys(" ")

clickable = driver.find_element(By.ID, 'login-signin')
clickable.click()

## Setup Functions

In [8]:
def schedule_generator():
    #run once at start of season to get schedule
    element = driver.find_element(By.ID, 'yui_3_18_1_2_1692980405385_2017')
                                  
    sched = pd.DataFrame(columns=['Week', 'Team', 'Opponent'])
    for k in range(1,15): #number of weeks in reg season less one
        if k != 1:
            element.click()
            time.sleep(1.5)
        for i in range(1,8): #number of teams divided by two plus one (12/6 + 1 = 7)
                x1 = driver.find_element(By.XPATH, "/html/body/div[1]/div[2]/div[2]/div[2]/div/div/div[2]/div[2]/section[1]/div/div/div[3]/section[3]/div/section/ul/li[" +str(i) + "]/div[2]/div[1]/div/div[1]/div/div/div[1]/a").text
                x2 = driver.find_element(By.XPATH, "/html/body/div[1]/div[2]/div[2]/div[2]/div/div/div[2]/div[2]/section[1]/div/div/div[3]/section[3]/div/section/ul/li[" +str(i) + "]/div[2]/div[3]/div/div[2]/div/div/div[1]/a").text
                newrow = {'Week': k, 'Team': x1, 'Opponent': x2}

                sched = sched.append(newrow, ignore_index = True)
    
    sched.to_csv('OneUSAC2023_sched.csv')
    return 0

#schedule_generator()



In [9]:
# CHECK NUMBERS AND REVISE

avg = 120 #pulled from previous season results 
wobble = 20 #revised upwards slightly

def score_generator(team, week):
    try:
        mu = remaining_table.loc[team][week]
        predict = np.random.normal(loc=mu, scale=wobble)
        return(predict)
    except: #bye weeks (in playoffs)
        return(0)

#demo of score functions
for k in range(0, 51, 5):
    j = 0
    for i in range(0,100000):
        x = np.random.normal(loc=1157+k, scale=wobble)
        y = np.random.normal(loc=1157, scale=wobble)
        if x > y:
            j = j + 1
    print(k, ' ', j/100000)

0   0.49935
5   0.56907
10   0.63694
15   0.70092
20   0.75829
25   0.81167
30   0.85723
35   0.89127
40   0.92155
45   0.94423
50   0.9606


## Score Frame + Seed Table

In [10]:
for wk in range(1,wk_count+1):
    league_id = league_url + '/matchup?week=' + str(wk) + '&mid1=7&mid2=6'
    driver.get(league_id)
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(5)
    
    tuples = []
    for i in range(1,8): #number of teams div 2 plus one
        y1 =driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div[2]/div/div/div[2]/div[2]/section/div/div/div[3]/section[3]/div/section/ul/li[' + str(i) + ']/div[2]/div[1]/div/div[1]/div/div/div[1]/a').text
        z1 =driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div[2]/div/div/div[2]/div[2]/section/div/div/div[3]/section[3]/div/section/ul/li[' + str(i) + ']/div[2]/div[1]/div/div[2]/div/div[1]').text
        
        y2 =driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div[2]/div/div/div[2]/div[2]/section/div/div/div[3]/section[3]/div/section/ul/li[' + str(i) + ']/div[2]/div[3]/div/div[2]/div/div/div[1]/a').text
        z2 =driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div[2]/div/div/div[2]/div[2]/section/div/div/div[3]/section[3]/div/section/ul/li[' + str(i) + ']/div[2]/div[3]/div/div[1]/div/div[1]').text
            
        if float(z1) > float(z2):
            w1 = 1
            w2 = 0
        if float(z1) == float(z2):
            w1 = 0
            w2 = 0
        else:
            w1 = 0
            w2 = 1
            
        item = tuple([y1,z1,w1])
        tuples.append(item)
        item = tuple([y2,z2,w2])
        tuples.append(item)
    
    tuples.sort(key = lambda x: x[0])  
    
    if wk == 1:
        scores = pd.DataFrame(columns=['team'], data=[x[0] for x in tuples])
        scores['W'] = [0] * len(scores)
     
    scores[wk] = [float(x[1]) for x in tuples]
    scores['W'] = scores.W + [int(x[2]) for x in tuples]
    
scores = scores.set_index('team')

scores['totals'] = scores.drop(['W'],axis=1).sum(axis=1)

seed_table = scores[['totals', 'W']]
seed_table.columns = ['Points', 'W']

wk_list = [i for i in range(1,wk_count+1)]
wk_list.insert(0, 'totals')

scores = scores[wk_list]

In [11]:
seed_table #current win count and point total

,Points,W
team,,
#CorruptCommish,0.0,0
5 and Juan,0.0,0
ACE of L.A.D.,0.0,0
CTE Pros (not concussions-SQL),0.0,0
Chase's Superb Team,0.0,0
Chip Kelly,0.0,0
Erik's Peerless Team,0.0,0
Fool Me 100 Times Shame On Me,0.0,0
GhostOfRayLewis,0.0,0


In [12]:
for i in range (wk_count+1, 18): #17 week season
    scores[i] = np.nan
    
scores

,totals,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
team,,,,,,,,,,,,,,,,,,
#CorruptCommish,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5 and Juan,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ACE of L.A.D.,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CTE Pros (not concussions-SQL),0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Chase's Superb Team,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Chip Kelly,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Erik's Peerless Team,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fool Me 100 Times Shame On Me,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GhostOfRayLewis,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
bye_multiplier = [1, 1, 1, 1,
                 1.143, 1.067, 1.231, 1,
                 1.143, 1.143, 1.143, 1,
                 1.231, 1.067, 1, 1, 1] #2023 nfl season byes

i = 1

for team in scores.index:
    print(team)
    final_scores = list(scores.loc[team][1:wk_count+1])
    try:
        neg_one = final_scores[-1] * bye_multiplier[wk_count-1]
    except IndexError:
        if wk_count >= 1:
            neg_one = avg * bye_multiplier[wk_count-1]
        else:
            neg_one = avg
        
    try:
        neg_two = final_scores[-2] * bye_multiplier[wk_count-2]
    except IndexError:
        if wk_count >= 2:
            neg_two = avg * bye_multiplier[wk_count-2]
        else:
            neg_two = avg
        
    try:
        neg_three = final_scores[-3] * bye_multiplier[wk_count-3]
    except IndexError:
        if wk_count >= 3:
            neg_three = avg * bye_multiplier[wk_count-3]
        else:
            neg_three = avg
        
    try:
        neg_four = final_scores[-4] * bye_multiplier[wk_count-4]
    except IndexError:
        if wk_count >= 4:
            neg_four = avg * bye_multiplier[wk_count-4]
        else:
            neg_four = avg
        
    try:
        neg_five = final_scores[-5] * bye_multiplier[wk_count-5]
    except IndexError:
        if wk_count >= 5:
            neg_five = avg * bye_multiplier[wk_count-5]
        else:
            neg_five = avg
    
    try:
        plus_one = ((neg_one+neg_two+neg_three+neg_four+neg_five)/5) * (1/bye_multiplier[wk_count])
        plus_two = ((neg_one+neg_two+neg_three+neg_four+avg)/5) * (1/bye_multiplier[wk_count+1])
        plus_three = ((neg_one+neg_two+neg_three+avg+avg)/5) * (1/bye_multiplier[wk_count+2])
        plus_four = ((neg_one+neg_two+avg+avg+avg)/5) * (1/bye_multiplier[wk_count+3])
        plus_five = ((neg_one+avg+avg+avg+avg)/5) * (1/bye_multiplier[wk_count+4])
        
        for item in [plus_one, plus_two, plus_three, plus_four, plus_five]:
            final_scores.append(item)

    
    except IndexError:
        try:
            plus_one = ((neg_one+neg_two+neg_three+neg_four+neg_five)/5) * (1/bye_multiplier[wk_count])
            plus_two = ((neg_one+neg_two+neg_three+neg_four+avg)/5) * (1/bye_multiplier[wk_count+1])
            plus_three = ((neg_one+neg_two+neg_three+avg+avg)/5) * (1/bye_multiplier[wk_count+2])
            plus_four = ((neg_one+neg_two+avg+avg+avg)/5) * (1/bye_multiplier[wk_count+3])

            for item in [plus_one, plus_two, plus_three, plus_four]:
                final_scores.append(item)
        

        except IndexError:
            plus_one = ((neg_one+neg_two+neg_three+neg_four+neg_five)/5) * (1/bye_multiplier[wk_count])
            plus_two = ((neg_one+neg_two+neg_three+neg_four+avg)/5) * (1/bye_multiplier[wk_count+1])
            plus_three = ((neg_one+neg_two+neg_three+avg+avg)/5) * (1/bye_multiplier[wk_count+2])
    
    i = wk_count
                
    for item in [plus_one, plus_two, plus_three]:
        final_scores.append(item)
    
    
    while len(final_scores) < 17:
        final_scores.append(round(avg * (1/bye_multiplier[i+5]), 3))
        i = i + 1
    
    final_scores = [round(i, 2) for i in final_scores]
    scores.loc[team, 1:18] = final_scores

#CorruptCommish
5 and Juan
ACE of L.A.D.
CTE Pros (not concussions-SQL)
Chase's Superb Team
Chip Kelly
Erik's Peerless Team
Fool Me 100 Times Shame On Me
GhostOfRayLewis
Leading the League in USACs
N888888888888888888
Rod the Great's Team
The Dream Team
The Teamwork Encouragers


C:\Users\Zach Gozlan\Anaconda3\lib\site-packages\ipykernel_launcher.py:89: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.


In [69]:
scores

,totals,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
team,,,,,,,,,,,,,,,,,,
#CorruptCommish,0.0,0.0,96.0,96.0,96.0,83.99,89.97,96.0,96.0,96.0,97.48,120.0,104.99,104.99,104.99,120.0,97.48,112.47
5 and Juan,0.0,0.0,96.0,96.0,96.0,83.99,89.97,96.0,96.0,96.0,97.48,120.0,104.99,104.99,104.99,120.0,97.48,112.47
ACE of L.A.D.,0.0,0.0,96.0,96.0,96.0,83.99,89.97,96.0,96.0,96.0,97.48,120.0,104.99,104.99,104.99,120.0,97.48,112.47
CTE Pros (not concussions-SQL),0.0,0.0,96.0,96.0,96.0,83.99,89.97,96.0,96.0,96.0,97.48,120.0,104.99,104.99,104.99,120.0,97.48,112.47
Chase's Superb Team,0.0,0.0,96.0,96.0,96.0,83.99,89.97,96.0,96.0,96.0,97.48,120.0,104.99,104.99,104.99,120.0,97.48,112.47
Chip Kelly,0.0,0.0,96.0,96.0,96.0,83.99,89.97,96.0,96.0,96.0,97.48,120.0,104.99,104.99,104.99,120.0,97.48,112.47
Erik's Peerless Team,0.0,0.0,96.0,96.0,96.0,83.99,89.97,96.0,96.0,96.0,97.48,120.0,104.99,104.99,104.99,120.0,97.48,112.47
Fool Me 100 Times Shame On Me,0.0,0.0,96.0,96.0,96.0,83.99,89.97,96.0,96.0,96.0,97.48,120.0,104.99,104.99,104.99,120.0,97.48,112.47
GhostOfRayLewis,0.0,0.0,96.0,96.0,96.0,83.99,89.97,96.0,96.0,96.0,97.48,120.0,104.99,104.99,104.99,120.0,97.48,112.47


In [15]:
remaining_sched = pd.read_csv('OneUSAC2023_sched.csv') #generated from schedule thing above
remaining_sched = remaining_sched[remaining_sched['Week'] > wk_count]
remaining_sched.head(12)

,Week,Team,Opponent
7,2,The Teamwork Encouragers,GhostOfRayLewis
8,2,#CorruptCommish,Erik's Peerless Team
9,2,5 and Juan,Leading the League in USACs
10,2,Fool Me 100 Times Shame On Me,CTE Pros (not concussions-SQL)
11,2,The Dream Team,Rod the Great's Team
12,2,Chip Kelly,Chase's Superb Team
13,2,ACE of L.A.D.,N888888888888888888
14,3,The Teamwork Encouragers,Erik's Peerless Team
15,3,#CorruptCommish,The Dream Team
16,3,5 and Juan,CTE Pros (not concussions-SQL)


In [16]:
seed_table
remaining_table = scores.drop(['totals'], axis=1)

In [68]:
#this week

def score_generator(team, week):
    try:
        mu = remaining_table.loc[team][week]
        predict = np.random.normal(loc=mu, scale=wobble)
        return(predict)
    except: #bye weeks (in playoffs)
        return(0)

def score_generator2(team, week):
    try:
        mu = remaining_table.loc[team][week]
        predict = np.random.normal(loc=mu, scale=wobble)
        return(mu)
    except: #bye weeks (in playoffs)
        return(0)

def prob_10k(score1, score2):
    j = 0
    for i in range(0,100000):
        x = np.random.normal(loc=score1, scale=wobble)
        y = np.random.normal(loc=score2, scale=wobble)
        if x > y:
            j = j + 1
    return(j/100000)

#displays current week's head to head win probabilities

remaining_sched['TeamScore'] = [score_generator2(j.Team, j.Week) for i,j in remaining_sched.iterrows()]
remaining_sched['OppoScore'] = [score_generator2(j.Opponent, j.Week) for i,j in remaining_sched.iterrows()]
remaining_sched['team_win'] = [int(j.TeamScore>=j.OppoScore) for i,j in remaining_sched.iterrows()] #there won't be ties
remaining_sched['oppo_win'] = [int(j.TeamScore<j.OppoScore) for i,j in remaining_sched.iterrows()]
remaining_sched['prob'] = [prob_10k(j.TeamScore, j.OppoScore) for i,j in remaining_sched.iterrows()]

remaining_sched.head(6)

,Week,Team,Opponent,TeamScore,OppoScore,team_win,oppo_win,prob
7,2,The Teamwork Encouragers,GhostOfRayLewis,96.0,96.0,1,0,0.50143
8,2,#CorruptCommish,Erik's Peerless Team,96.0,96.0,1,0,0.49814
9,2,5 and Juan,Leading the League in USACs,96.0,96.0,1,0,0.49864
10,2,Fool Me 100 Times Shame On Me,CTE Pros (not concussions-SQL),96.0,96.0,1,0,0.50172
11,2,The Dream Team,Rod the Great's Team,96.0,96.0,1,0,0.49795
12,2,Chip Kelly,Chase's Superb Team,96.0,96.0,1,0,0.50176


## One Loop

The below cells run through one loop of the final output; the season is played out based on the probabilities generated above and the remaining schedule.

In [18]:
remaining_sched['TeamScore'] = [score_generator(j.Team, j.Week) for i,j in remaining_sched.iterrows()]
remaining_sched['OppoScore'] = [score_generator(j.Opponent, j.Week) for i,j in remaining_sched.iterrows()]
remaining_sched['team_win'] = [int(j.TeamScore>=j.OppoScore) for i,j in remaining_sched.iterrows()] #there won't be ties
remaining_sched['oppo_win'] = [int(j.TeamScore<j.OppoScore) for i,j in remaining_sched.iterrows()]
remaining_sched.head(6)

,Week,Team,Opponent,TeamScore,OppoScore,team_win,oppo_win,prob
7,2,The Teamwork Encouragers,GhostOfRayLewis,105.616077,101.918194,1,0,0.50024
8,2,#CorruptCommish,Erik's Peerless Team,59.496337,94.178008,0,1,0.49686
9,2,5 and Juan,Leading the League in USACs,126.277923,78.976601,1,0,0.50045
10,2,Fool Me 100 Times Shame On Me,CTE Pros (not concussions-SQL),70.585917,89.006204,0,1,0.49887
11,2,The Dream Team,Rod the Great's Team,85.485799,114.301508,0,1,0.50019
12,2,Chip Kelly,Chase's Superb Team,103.377286,113.237706,0,1,0.49719


In [19]:
#simplify that frame down to the number of wins each team gains in that simmed run of the league

wins1 = remaining_sched[['Team', 'team_win', 'TeamScore']]
wins2 = remaining_sched[['Opponent', 'oppo_win', 'OppoScore']]
wins1 = wins1.groupby('Team').sum()
wins2 = wins2.groupby('Opponent').sum()
wins2 = wins2.rename(columns={'oppo_win':'team_win'})

wins3 = pd.concat([wins1, wins2])
wins3['Team'] = wins3.index
simmed_wins = wins3.groupby('Team').sum()
simmed_wins['Points'] = simmed_wins.OppoScore + simmed_wins.TeamScore
simmed_wins = simmed_wins.drop(['OppoScore', 'TeamScore'], axis=1)

simmed_wins

,team_win,Points
Team,,
#CorruptCommish,7,1372.032020
5 and Juan,5,1258.704310
ACE of L.A.D.,7,1268.453300
CTE Pros (not concussions-SQL),7,1263.061093
Chase's Superb Team,7,1249.505834
Chip Kelly,8,1334.145128
Erik's Peerless Team,7,1245.681225
Fool Me 100 Times Shame On Me,4,1310.901494
GhostOfRayLewis,7,1263.945278


In [20]:
#read in the current standings frame

current_standings = seed_table
current_standings = current_standings[['Points', 'W']]
new_standings = current_standings

new_standings

,Points,W
team,,
#CorruptCommish,0.0,0
5 and Juan,0.0,0
ACE of L.A.D.,0.0,0
CTE Pros (not concussions-SQL),0.0,0
Chase's Superb Team,0.0,0
Chip Kelly,0.0,0
Erik's Peerless Team,0.0,0
Fool Me 100 Times Shame On Me,0.0,0
GhostOfRayLewis,0.0,0


In [21]:
#add the simmed wins to the current wins to get a simmed "outcome" of the league

#for team in scores.index:
#    new_standings.at[team, 'W'] = new_standings.at[team, 'W'] + simmed_wins.at[team, 'team_win']

#final week only commented out

In [22]:
new_standings

,Points,W
team,,
#CorruptCommish,0.0,0
5 and Juan,0.0,0
ACE of L.A.D.,0.0,0
CTE Pros (not concussions-SQL),0.0,0
Chase's Superb Team,0.0,0
Chip Kelly,0.0,0
Erik's Peerless Team,0.0,0
Fool Me 100 Times Shame On Me,0.0,0
GhostOfRayLewis,0.0,0


In [23]:
#rearrange the simulated outcome frame by wins and then points (the tiebreaker metric) to determine the "place" of each team
#in the simmed league

simmed_standings = current_standings.sort_values(by=['W', 'Points'], ascending=False)
simmed_standings['Points'] = current_standings['Points'] #+ simmed_wins['Points'] final week only
simmed_standings = simmed_standings.sort_values(by=['W', 'Points'], ascending=False)
simmed_standings['place'] = np.arange(1,15)

In [24]:
simmed_standings

,Points,W,place
team,,,
#CorruptCommish,0.0,0,1
5 and Juan,0.0,0,2
ACE of L.A.D.,0.0,0,3
CTE Pros (not concussions-SQL),0.0,0,4
Chase's Superb Team,0.0,0,5
Chip Kelly,0.0,0,6
Erik's Peerless Team,0.0,0,7
Fool Me 100 Times Shame On Me,0.0,0,8
GhostOfRayLewis,0.0,0,9


In [25]:
simmed_standings['Team'] = simmed_standings.index

simmed_standings = simmed_standings[['Team', 'Points', 'W', 'place']]

x = pd.DataFrame.to_numpy(simmed_standings)

y = pd.DataFrame(x, columns=['Team', 'Points', 'W', 'place'])
y = y.set_index('Team')

y

,Points,W,place
Team,,,
#CorruptCommish,0,0,1
5 and Juan,0,0,2
ACE of L.A.D.,0,0,3
CTE Pros (not concussions-SQL),0,0,4
Chase's Superb Team,0,0,5
Chip Kelly,0,0,6
Erik's Peerless Team,0,0,7
Fool Me 100 Times Shame On Me,0,0,8
GhostOfRayLewis,0,0,9


In [26]:
team_list = []
points_list = []
w_list = []
place_list = []

In [27]:
team_list.extend(y.index)
points_list.extend(y.Points)
w_list.extend(y.W)
place_list.extend(y.place)

In [28]:
array = pd.DataFrame()
array['team'] = team_list
array['Points'] = points_list
array['W'] = w_list
array['place'] = place_list

array

,team,Points,W,place
0,#CorruptCommish,0.0,0,1
1,5 and Juan,0.0,0,2
2,ACE of L.A.D.,0.0,0,3
3,CTE Pros (not concussions-SQL),0.0,0,4
4,Chase's Superb Team,0.0,0,5
5,Chip Kelly,0.0,0,6
6,Erik's Peerless Team,0.0,0,7
7,Fool Me 100 Times Shame On Me,0.0,0,8
8,GhostOfRayLewis,0.0,0,9
9,Leading the League in USACs,0.0,0,10


In [29]:
dict_test = simmed_standings[['place']].to_dict()
dict2 = dict_test.pop('place')
inv_map = {v: k for k, v in dict2.items()}
inv_map

inv_map.update( {15 : 'bye'} )

print('Seeds')
print(inv_map)
print('\n')

matchups_r1 = {'Team': [inv_map[1], inv_map[4], inv_map[3], inv_map[2], inv_map[7],inv_map[10], inv_map[9], inv_map[8]],
        'Opponent': [inv_map[15], inv_map[5], inv_map[6], inv_map[15], inv_map[15],inv_map[11], inv_map[12], inv_map[15]]
        }

playoff_r1 = pd.DataFrame(matchups_r1, columns = ['Team', 'Opponent'])

playoff_r1

Seeds
{1: '#CorruptCommish', 2: '5 and Juan', 3: 'ACE of L.A.D.', 4: 'CTE Pros (not concussions-SQL)', 5: "Chase's Superb Team", 6: 'Chip Kelly', 7: "Erik's Peerless Team", 8: 'Fool Me 100 Times Shame On Me', 9: 'GhostOfRayLewis', 10: 'Leading the League in USACs', 11: 'N888888888888888888', 12: "Rod the Great's Team", 13: 'The Dream Team', 14: 'The Teamwork Encouragers', 15: 'bye'}




,Team,Opponent
0,#CorruptCommish,bye
1,CTE Pros (not concussions-SQL),Chase's Superb Team
2,ACE of L.A.D.,Chip Kelly
3,5 and Juan,bye
4,Erik's Peerless Team,bye
5,Leading the League in USACs,N888888888888888888
6,GhostOfRayLewis,Rod the Great's Team
7,Fool Me 100 Times Shame On Me,bye


In [30]:
dict_test = simmed_standings[['place']].to_dict()
dict2 = dict_test.pop('place')
inv_map = {v: k for k, v in dict2.items()}
inv_map

inv_map.update( {15 : 'bye'} )

print('Seeds')
print(inv_map)
print('\n')

matchups_r1 = {'Team': [inv_map[1], inv_map[4], inv_map[3], inv_map[2], inv_map[7],inv_map[10], inv_map[9], inv_map[8]],
        'Opponent': [inv_map[15], inv_map[5], inv_map[6], inv_map[15], inv_map[15],inv_map[11], inv_map[12], inv_map[15]]
        }

playoff_r1 = pd.DataFrame(matchups_r1, columns = ['Team', 'Opponent'])

print('Round 1')

playoff_r1['TeamScore'] = [score_generator(j.Team, 15) for i,j in playoff_r1.iterrows()]
playoff_r1['OppoScore'] = [score_generator(j.Opponent, 15) for i,j in playoff_r1.iterrows()]
playoff_r1['team_win'] = [int(j.TeamScore>=j.OppoScore) for i,j in playoff_r1.iterrows()] #there won't be ties
playoff_r1['oppo_win'] = [int(j.TeamScore<j.OppoScore) for i,j in playoff_r1.iterrows()]

winners_r1 = []
losers_r1 = []
for index, row in playoff_r1.iterrows():
    if row['team_win'] == 1:
        winners_r1.append(row['Team'])
    else:
        winners_r1.append(row['Opponent'])
        
print(winners_r1)
print('\n')

matchups_r2 = {'Team': [winners_r1[0], winners_r1[2], winners_r1[4], winners_r1[6]], 
        'Opponent': [winners_r1[1], winners_r1[3], winners_r1[5], winners_r1[7]]
        }

playoff_r2 = pd.DataFrame(matchups_r2, columns = ['Team', 'Opponent'])

playoff_r2['TeamScore'] = [score_generator(j.Team, 16) for i,j in playoff_r2.iterrows()]
playoff_r2['OppoScore'] = [score_generator(j.Opponent, 16) for i,j in playoff_r2.iterrows()]
playoff_r2['team_win'] = [int(j.TeamScore>=j.OppoScore) for i,j in playoff_r2.iterrows()] #there won't be ties
playoff_r2['oppo_win'] = [int(j.TeamScore<j.OppoScore) for i,j in playoff_r2.iterrows()]

winners_r2 = []
losers_r2 = []
for index, row in playoff_r2.iterrows():
    if row['team_win'] == 1:
        winners_r2.append(row['Team'])
        losers_r2.append(row['Opponent'])
    else:
        winners_r2.append(row['Opponent'])
        losers_r2.append(row['Team'])

print('Round 2')
print(winners_r2)
print('\n')

matchups_r3 = {'Team': [winners_r2[0], losers_r2[0], winners_r2[2]], 
        'Opponent': [winners_r2[1], losers_r2[1], winners_r2[3]]
        }

playoff_r3 = pd.DataFrame(matchups_r3, columns = ['Team', 'Opponent'])

playoff_r3['TeamScore'] = [score_generator(j.Team, 17) for i,j in playoff_r3.iterrows()]
playoff_r3['OppoScore'] = [score_generator(j.Opponent, 17) for i,j in playoff_r3.iterrows()]
playoff_r3['team_win'] = [int(j.TeamScore>=j.OppoScore) for i,j in playoff_r3.iterrows()] #there won't be ties
playoff_r3['oppo_win'] = [int(j.TeamScore<j.OppoScore) for i,j in playoff_r3.iterrows()]

winners_r3 = []
losers_r3 = []
for index, row in playoff_r3.iterrows():
    if row['team_win'] == 1:
        winners_r3.append(row['Team'])
        losers_r3.append(row['Opponent'])
    else:
        winners_r3.append(row['Opponent'])
        losers_r3.append(row['Team'])
        
print('Round 3')
print(winners_r3)
print('\n')

Seeds
{1: '#CorruptCommish', 2: '5 and Juan', 3: 'ACE of L.A.D.', 4: 'CTE Pros (not concussions-SQL)', 5: "Chase's Superb Team", 6: 'Chip Kelly', 7: "Erik's Peerless Team", 8: 'Fool Me 100 Times Shame On Me', 9: 'GhostOfRayLewis', 10: 'Leading the League in USACs', 11: 'N888888888888888888', 12: "Rod the Great's Team", 13: 'The Dream Team', 14: 'The Teamwork Encouragers', 15: 'bye'}


Round 1
['#CorruptCommish', "Chase's Superb Team", 'Chip Kelly', '5 and Juan', "Erik's Peerless Team", 'N888888888888888888', 'GhostOfRayLewis', 'Fool Me 100 Times Shame On Me']


Round 2
["Chase's Superb Team", 'Chip Kelly', "Erik's Peerless Team", 'GhostOfRayLewis']


Round 3
['Chip Kelly', '5 and Juan', "Erik's Peerless Team"]




In [31]:
champ_table = pd.DataFrame(columns=['First', 'Second', 'Third', 'Consolation'])

matchups = []
for index, row in champ_table.iterrows():
    championship = []
    championship.append(row['First'])
    championship.append(row['Second'])
    championship = sorted(championship)
    matchups.append(str(championship))
    
counter=collections.Counter(matchups)
counter

champ_table = champ_table.append({'First' : winners_r3[0], 'Second' : losers_r3[0], 'Third' : winners_r3[1], 'Consolation' : winners_r3[2] } , ignore_index=True)

champ_table

,First,Second,Third,Consolation
0,Chip Kelly,Chase's Superb Team,5 and Juan,Erik's Peerless Team


## Does The Deal

Runs the above loop *number_sims* times, usually 10,000 (~20 minute runtime).

In [32]:
#this is the equivalent of looping the previous five code blocks n number of times to get a distribution of possible outcomes
#for each team

number_sims = 10000 #one hour for full schedule at 10k sims
#all_predictions = pd.DataFrame(columns=['Points', 'W', 'place'])
i = 0

team_list = []
points_list = []
w_list = []
place_list = []


first_list = []
second_list = []
third_list = []
consolation_list = []

start = time.time()

while i < number_sims: 
    #game sims
    
    current_standings = seed_table.copy()
    
    remaining_table = scores.drop(['totals'], axis=1) #automate this
    
    remaining_sched['TeamScore'] = [score_generator(j.Team, j.Week) for i,j in remaining_sched.iterrows()]
    remaining_sched['OppoScore'] = [score_generator(j.Opponent, j.Week) for i,j in remaining_sched.iterrows()]
    remaining_sched['team_win'] = [int(j.TeamScore>j.OppoScore) for i,j in remaining_sched.iterrows()]
    remaining_sched['oppo_win'] = [int(j.TeamScore<j.OppoScore) for i,j in remaining_sched.iterrows()]
    
    wins1 = remaining_sched[['Team', 'team_win', 'TeamScore']]
    wins2 = remaining_sched[['Opponent', 'oppo_win', 'OppoScore']]
    wins1 = wins1.groupby('Team').sum()
    wins2 = wins2.groupby('Opponent').sum()
    wins2 = wins2.rename(columns={'oppo_win':'team_win'})

    wins3 = pd.concat([wins1, wins2])
    wins3['Team'] = wins3.index
    simmed_wins = wins3.groupby('Team').sum()
    simmed_wins['Points'] = simmed_wins.OppoScore + simmed_wins.TeamScore
    simmed_wins = simmed_wins.drop(['OppoScore', 'TeamScore'], axis=1)
    
    
    #FINAL WEEK: UNCOMMENT THIS CODE AND COMMENT OUT LINES 49/50
    #new_standings = current_standings[['Points', 'W']]
    
    for team in team_list[:14]:
        new_standings.at[team, 'W'] = new_standings.at[team, 'W'] + simmed_wins.at[team, 'team_win']
    
     
    #rearrange the simulated outcome frame by wins and then points (the tiebreaker metric) to determine the "place" of each team
    #in the simmed league

    simmed_standings = new_standings.sort_values(by=['W', 'Points'], ascending=False)

    #a random "points" jigger for remaining weeks to account for win ties
    simmed_standings['Points'] = new_standings['Points']# + simmed_wins['Points'] final week commented out

    simmed_standings = simmed_standings.sort_values(by=['W', 'Points'], ascending=False)
    #ranking assignment
    simmed_standings['place'] = np.arange(1,15)
    
    #all_predictions = pd.concat([all_predictions, simmed_standings])
    
    team_list.extend(simmed_standings.index)
    points_list.extend(simmed_standings.Points)
    w_list.extend(simmed_standings.W)
    place_list.extend(simmed_standings.place)
    
    ## PLAYOFF SIMS
    
    dict_test = simmed_standings[['place']].to_dict()
    dict2 = dict_test.pop('place')
    inv_map = {v: k for k, v in dict2.items()}
    inv_map

    inv_map.update( {15 : 'bye'} )
    
    matchups_r1 = {'Team': [inv_map[1], inv_map[4], inv_map[3], inv_map[2], inv_map[7],inv_map[10], inv_map[9], inv_map[8]],
        'Opponent': [inv_map[15], inv_map[5], inv_map[6], inv_map[15], inv_map[15],inv_map[11], inv_map[12], inv_map[15]]
        }

    playoff_r1 = pd.DataFrame(matchups_r1, columns = ['Team', 'Opponent'])

    playoff_r1['TeamScore'] = [score_generator(j.Team, 21) for i,j in playoff_r1.iterrows()]
    playoff_r1['OppoScore'] = [score_generator(j.Opponent, 21) for i,j in playoff_r1.iterrows()]
    playoff_r1['team_win'] = [int(j.TeamScore>=j.OppoScore) for i,j in playoff_r1.iterrows()] #there won't be ties
    playoff_r1['oppo_win'] = [int(j.TeamScore<j.OppoScore) for i,j in playoff_r1.iterrows()]

    winners_r1 = []
    losers_r1 = []
    for index, row in playoff_r1.iterrows():
        if row['team_win'] == 1:
            winners_r1.append(row['Team'])
        else:
            winners_r1.append(row['Opponent'])

    matchups_r2 = {'Team': [winners_r1[0], winners_r1[2], winners_r1[4], winners_r1[6]], 
            'Opponent': [winners_r1[1], winners_r1[3], winners_r1[5], winners_r1[7]]
            }

    playoff_r2 = pd.DataFrame(matchups_r2, columns = ['Team', 'Opponent'])

    playoff_r2['TeamScore'] = [score_generator(j.Team, 22) for i,j in playoff_r2.iterrows()]
    playoff_r2['OppoScore'] = [score_generator(j.Opponent, 22) for i,j in playoff_r2.iterrows()]
    playoff_r2['team_win'] = [int(j.TeamScore>=j.OppoScore) for i,j in playoff_r2.iterrows()] #there won't be ties
    playoff_r2['oppo_win'] = [int(j.TeamScore<j.OppoScore) for i,j in playoff_r2.iterrows()]

    winners_r2 = []
    losers_r2 = []
    for index, row in playoff_r2.iterrows():
        if row['team_win'] == 1:
            winners_r2.append(row['Team'])
            losers_r2.append(row['Opponent'])
        else:
            winners_r2.append(row['Opponent'])
            losers_r2.append(row['Team'])

    matchups_r3 = {'Team': [winners_r2[0], losers_r2[0], winners_r2[2]], 
            'Opponent': [winners_r2[1], losers_r2[1], winners_r2[3]]
            }

    playoff_r3 = pd.DataFrame(matchups_r3, columns = ['Team', 'Opponent'])

    playoff_r3['TeamScore'] = [score_generator(j.Team, 23) for i,j in playoff_r3.iterrows()]
    playoff_r3['OppoScore'] = [score_generator(j.Opponent, 23) for i,j in playoff_r3.iterrows()]
    playoff_r3['team_win'] = [int(j.TeamScore>=j.OppoScore) for i,j in playoff_r3.iterrows()] #there won't be ties
    playoff_r3['oppo_win'] = [int(j.TeamScore<j.OppoScore) for i,j in playoff_r3.iterrows()]

    winners_r3 = []
    losers_r3 = []
    for index, row in playoff_r3.iterrows():
        if row['team_win'] == 1:
            winners_r3.append(row['Team'])
            losers_r3.append(row['Opponent'])
        else:
            winners_r3.append(row['Opponent'])
            losers_r3.append(row['Team'])
            
    matchups = []
    '''for index, row in champ_table.iterrows():
        championship = []
        championship.append(row['First'])
        championship.append(row['Second'])
        championship = sorted(championship)
        matchups.append(str(championship))'''

    counter=collections.Counter(matchups)
    counter

    first_list.append(winners_r3[0])
    second_list.append(losers_r3[0])
    third_list.append(winners_r3[1])
    consolation_list.append(winners_r3[2])

    i = i + 1
    if (i % 1000 == 0):
        marker = time.time()
        print(str(i/100) + '%') #totally unnecessary, just gives a sense of progress
        print(marker - start)
        

10.0%
304.75862550735474


KeyboardInterrupt: 

In [33]:
array = pd.DataFrame()
array['team'] = team_list
array['Points'] = points_list
array['W'] = w_list
array['place'] = place_list

all_predictions = array.set_index('team')

all_predictions

,Points,W,place
team,,,
#CorruptCommish,0.0,0,1
5 and Juan,0.0,0,2
ACE of L.A.D.,0.0,0,3
CTE Pros (not concussions-SQL),0.0,0,4
Chase's Superb Team,0.0,0,5
...,...,...,...
ACE of L.A.D.,0.0,6773,10
Erik's Peerless Team,0.0,6770,11
#CorruptCommish,0.0,6766,12


In [34]:
champ_table = pd.DataFrame()

champ_table['First'] = first_list
champ_table['Second'] = second_list
champ_table['Third'] = third_list
champ_table['Consolation'] = consolation_list

In [35]:
first_table = pd.DataFrame(champ_table['First'].value_counts())
second_table = pd.DataFrame(champ_table['Second'].value_counts())
third_table = pd.DataFrame(champ_table['Third'].value_counts())
consolation_table = pd.DataFrame(champ_table['Consolation'].value_counts())

dfs = [first_table, second_table, third_table, consolation_table]

final = first_table.merge(second_table, how='outer', on=None, left_index=True, right_index=True)
final = final.merge(third_table, how='outer', on=None, left_index=True, right_index=True)
final = final.merge(consolation_table, how='outer', on=None, left_index=True, right_index=True)
final = final.fillna(0).sort_values(['First', 'Second', 'Third', 'Consolation'], ascending=False)
final['value'] = round((((final.First/10000)*175) + ((final.Second/10000)*100) + ((final.Third/10000)*50) + ((final.Consolation/10000)*25)),2)
final

,First,Second,Third,Consolation,value
CTE Pros (not concussions-SQL),214.0,25.0,16.0,109,4.35
GhostOfRayLewis,189.0,175.0,102.0,40,5.67
The Teamwork Encouragers,156.0,247.0,200.0,18,6.24
N888888888888888888,128.0,66.0,73.0,20,3.32
Leading the League in USACs,118.0,195.0,210.0,82,5.27
Chase's Superb Team,116.0,132.0,229.0,68,4.66
Fool Me 100 Times Shame On Me,56.0,34.0,53.0,114,1.87
#CorruptCommish,34.0,50.0,57.0,55,1.52
ACE of L.A.D.,20.0,108.0,64.0,342,2.61
5 and Juan,10.0,11.0,33.0,105,0.71


In [36]:
final.to_csv('OneUSAC_2023_playoffs_' + str(wk_count) + '.csv')

In [37]:
#table of distribution's of each team's simmed outcomes

outcome_dist = pd.pivot_table(array, values = ['place'], index = ['team'], columns=all_predictions.place.values, aggfunc='count', fill_value=0)

In [38]:
all_predictions['W2'] = [float(i) for i in array.W]
all_predictions['team'] = array.index
average_wins = pd.DataFrame(array.groupby('team')['W'].mean())
average_points = pd.DataFrame(array.groupby('team')['Points'].mean())

In [54]:
#number_sims = 1048

In [59]:
#season-ending placement probs

outcome_dist['Playoffs'] = (outcome_dist.place[1] + outcome_dist.place[2] 
                            + outcome_dist.place[3] + outcome_dist.place[4]
                            + outcome_dist.place[5] + outcome_dist.place[6]) / number_sims
outcome_dist['Consolation'] = (outcome_dist.place[7] + outcome_dist.place[8] 
                            + outcome_dist.place[9] + outcome_dist.place[10]
                            + outcome_dist.place[11] + outcome_dist.place[12]) / number_sims
outcome_dist['Bye'] = (outcome_dist.place[1] + outcome_dist.place[2]) / number_sims

outcome_dist['first'] = (outcome_dist.place[1]) / number_sims
outcome_dist['second'] = (outcome_dist.place[2]) / number_sims
outcome_dist['third'] = (outcome_dist.place[3]) / number_sims
outcome_dist['fourth'] = (outcome_dist.place[4]) / number_sims
outcome_dist['fifth'] = (outcome_dist.place[5]) / number_sims
outcome_dist['sixth'] = (outcome_dist.place[6]) / number_sims
outcome_dist['seventh'] = (outcome_dist.place[7]) / number_sims
outcome_dist['eighth'] = (outcome_dist.place[8]) / number_sims
outcome_dist['ninth'] = (outcome_dist.place[9]) / number_sims
outcome_dist['tenth'] = (outcome_dist.place[10]) / number_sims
outcome_dist['eleventh'] = (outcome_dist.place[11]) / number_sims
outcome_dist['twelfth'] = (outcome_dist.place[12]) / number_sims
outcome_dist['thirteenth'] = (outcome_dist.place[13]) / number_sims
outcome_dist['fourteenth'] = (outcome_dist.place[14]) / number_sims

outcome_dist = outcome_dist.sort_values(['first', 'second', 'third', 'fourth', 'fifth', 'sixth', 'seventh', 'eighth', 'ninth', 'tenth', 'eleventh', 'twelfth','thirteenth','fourteenth'], ascending=False) #after 20
outcome2 = outcome_dist.drop(['first', 'second', 'third', 'fourth', 'fifth', 'sixth', 'seventh', 'eighth', 'ninth', 'tenth', 'eleventh', 'twelfth','thirteenth','fourteenth'], axis=1) 
outcome3 = pd.merge(outcome2, average_wins, left_index=True, right_on='team')
outcome3 = pd.merge(outcome3, average_points, left_index=True, right_on='team')

outcome3.columns = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,'Playoffs','Consolation','Bye','Points','Wins']
points_col = array.groupby('team').mean()
#outcome3['AvgPoints'] = round(points_col.Points, 2)

outcome3

outcome3.sort_values(['Playoffs'], ascending=False)

C:\Users\Zach Gozlan\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,Playoffs,Consolation,Bye,Points,Wins
team,,,,,,,,,,,,,,,,,,,
Leading the League in USACs,118,205,195,210,94,73,82,36,22,9,0,1,3,0,0.854008,0.143130,0.308206,3431.237595,0.0
The Teamwork Encouragers,156,202,247,200,59,27,18,16,18,28,44,30,1,2,0.850191,0.146947,0.341603,3433.122137,0.0
GhostOfRayLewis,189,233,175,102,108,45,40,50,44,27,19,16,0,0,0.812977,0.187023,0.402672,3432.292939,0.0
Chase's Superb Team,116,85,132,229,152,88,68,75,39,29,10,16,9,0,0.765267,0.226145,0.191794,3426.329198,0.0
5 and Juan,10,5,11,33,228,277,105,17,42,48,79,108,85,0,0.538168,0.380725,0.014313,3408.736641,0.0
N888888888888888888,128,106,66,73,46,20,20,85,84,91,104,147,78,0,0.418893,0.506679,0.223282,3396.227099,0.0
CTE Pros (not concussions-SQL),214,86,25,16,33,40,109,139,180,70,27,58,38,13,0.395038,0.556298,0.286260,3412.739504,0.0
ACE of L.A.D.,20,42,108,64,80,86,342,144,68,44,21,22,7,0,0.381679,0.611641,0.059160,3408.620229,0.0
Fool Me 100 Times Shame On Me,56,24,34,53,105,93,114,277,181,48,2,0,61,0,0.348282,0.593511,0.076336,3402.837786,0.0


In [60]:
outcome3.sort_values(['Playoffs'], ascending=False).to_csv('OneUSAC2023_wk' + str(wk_count) + '.csv')

## Bonus: See R Code for viz output from this

In [63]:
for_viz = pd.DataFrame(columns=['team', 'Playoffs', 'Bye', 'week'])

for i in range(1,wk_count+1):
    path = 'OneUSAC2023_wk' + str(i) + '.csv'
    numbers = pd.read_csv(path)
    numbers = numbers[['team', 'Playoffs', 'Bye']]
    numbers['week'] = i
    for_viz = for_viz.append(numbers, ignore_index=True)
    
for_viz.to_csv('weekly_chart.csv')

In [64]:
save_grief = 'tmp$team <- factor(tmp$team, c("'

for i in for_viz['team']:
    save_grief += i
    save_grief += '","'
    
save_grief = save_grief[:-3]
save_grief += '"))'

save_grief

'tmp$team <- factor(tmp$team, c("Leading the League in USACs","The Teamwork Encouragers","GhostOfRayLewis","Chase\'s Superb Team","5 and Juan","N888888888888888888","CTE Pros (not concussions-SQL)","ACE of L.A.D.","Fool Me 100 Times Shame On Me","The Dream Team","#CorruptCommish","Rod the Great\'s Team","Chip Kelly","Erik\'s Peerless Team"))'